<a href="https://colab.research.google.com/github/tossy66/langchain-training/blob/main/chapter3/langchain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 0: ハンズオンの準備
---

## 必要なライブラリのインストール

In [1]:
# !pip install -q langchain langchain-core langchain-text-splitters

In [2]:
!pip install -q langchain-openai langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 879.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [4]:
!pip freeze | grep langchain

langchain==0.3.26
langchain-community==0.3.27
langchain-core==0.3.68
langchain-openai==0.3.28
langchain-text-splitters==0.3.8


## API キーの設定
*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `OPENAI_API_KEY` と入力し、その [**値**] に指定されたキーを入力します。
*  [**新しいシークレットを追加**] をクリックし、[**名前**] に `LANGCHAIN_API_KEY` と入力し、その [**値**] に LangSmith で作成してコピーしておいた API キーを入力します。
*  設定した 2 つのシークレットの [**ノートブックからのアクセス**] を有効にします。
*  入力が完了したら、下のセルを実行します。

In [5]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Section 1: LangChain の基本
---

## LLMs
https://python.langchain.com/docs/concepts/#llms  

[langchain_openai.llms.base.OpenAI](https://python.langchain.com/api_reference/openai/llms/langchain_openai.llms.base.OpenAI.html)

In [6]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

result = llm.invoke("自己紹介してください。")
print(result)



私は山田太郎と申します。東京出身で、現在は大学生として都内の大学に通っています。趣味は読書や映画鑑賞、旅行などです。また、大学では国際関係を専攻しており、将来は国際的な仕事に携わりたいと考えています。人とのコミュニケーションが得意で、チームでの活動やリーダーシップを取ることが好きです。今後も自分を磨き、社会に貢献できるよう努力していきたいと思っています。よろしくお願いします。




*   LLM モデルは文字列を入力として受け取り、文字列を出力する。
*   単純な一問一答の場合に使用する。
*   この程度であれば、LnagChain を使う意味はあまりない。



## Chat models
https://python.langchain.com/docs/concepts/#chat-models

[langchain_openai.chat_models.base.ChatOpenAI](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

In [8]:
messages = [
    SystemMessage(content="以下の日本語を英語に翻訳してください。"),
    HumanMessage(content="こんにちは!"),
]

response = model.invoke(messages)
response

AIMessage(content='Hello!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 25, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BuAO8qy7Uk8FYGojR2rMvf0PnR9K1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--4fb1e23d-23f4-48b6-a3d4-fd8734615df4-0', usage_metadata={'input_tokens': 25, 'output_tokens': 2, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

*   Chat Model は Message のリストを入力として受け取り、Message を出力する。
*   Message は `role` `content` `response_metadata` を属性として持つ。
*   LangChain の `SystemMessage` `HumanMessage` `AIMessage` は、それぞれ Chat Completions API の `role: "system"` `role: "user"` `role: "assistant"` に対応している。

```
{
  "model": "gpt-4o-mini"
  "messages": [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    {"role": "assistant": "content": "こんにちは、ジョンさん！どのようにお手伝いできますか？"},
    {"role": "user": "content": "私の名前がわかりますか？"}
  ],
  (省略)
}
```



### `response` の内容を表示する

In [9]:
print(response)

content='Hello!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 25, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BuAO8qy7Uk8FYGojR2rMvf0PnR9K1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--4fb1e23d-23f4-48b6-a3d4-fd8734615df4-0' usage_metadata={'input_tokens': 25, 'output_tokens': 2, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### `content` だけを表示する

In [10]:
print(response.content)

Hello!


### 会話の履歴を入力に含める

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = model.invoke(messages)
print(result.content)

はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいですか？




*   `AIMessage` を入力に含めることができる。
*   AI の回答も含めた過去の会話履歴を入力に含めることによって、会話を継続することができる (後述)。



OpenAI 以外にも、様々な Chat model を利用することができる。  
https://python.langchain.com/docs/integrations/chat/

## Prompt templates
プロンプトをテンプレート化しておき、ユーザーの入力で補完してプロンプトを作成する際に使用する。  
https://python.langchain.com/docs/concepts/prompt_templates/


### StringPromptTemplate
- LLM への単純な入力に使用する。  
- 単一の文字列としてプロンプトを作成する際に使用する。  
[langchain_core.prompts.prompt.PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html)

In [12]:
from langchain_core.prompts import PromptTemplate

In [13]:
prompt_template = PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名: {dish}""")

prompt = prompt_template.invoke({"dish": "カレー"})
prompt
# print(prompt.text)

StringPromptValue(text='以下の料理のレシピを考えてください。\n\n料理名: カレー')

In [14]:
result = model.invoke(prompt)
print(result.content)

もちろんです！以下は基本的なカレーのレシピです。具材やスパイスはお好みに応じてアレンジできます。

### 材料（4人分）

- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量
- ご飯: 適量（白ご飯やナンなど）

### 作り方

1. **下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを加えて中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら火を弱め、アクを取りながら約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら味を見て、必要に応じて塩や胡椒で調整します。

6. **仕上げ**:
   - お好みでガーリックパウダーや生姜を加えて風味を増します。火を止めて、少し冷ますと味がなじみます。

7. **盛り付け**:
   - ご飯を皿に盛り、その上にカレーをかけて完成です。お好みで福神漬けやらっきょうを添えても良いでしょう。

### アレンジのアイデア
- **野菜カレー**: 鶏肉の代わりに豆腐やお好きな野菜（ナス、ピーマン、カボチャなど）を使っても美味しいです。
- **スパイシーカレー**: カレー粉やチリパウダーを追加して、辛さを調整できます。
- **ココナッツカレー**: ココナッツミルクを加えると、まろやかで風味豊かなカレーになります。

お好みに合わせてアレンジして、ぜひ楽しんでください！


### ChatPromptTemplate
*  PromptTemplate を Chat model の形式に対応させたもの。  
*  SystemMessage、HumanMessage、AIMessage をそれぞれテンプレート化して、ChatPromptTemplate というクラスでまとめて扱うことができる。

In [15]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [16]:
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："),
    HumanMessagePromptTemplate.from_template("{question}")
])

messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください"})
# messages
print(messages)

messages=[SystemMessage(content='あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：', additional_kwargs={}, response_metadata={}), HumanMessage(content='簡単な Hello World のコードを作成してください', additional_kwargs={}, response_metadata={})]


以下のように書くこともできる。

In [17]:
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください。"})
messages
# print(messages)

ChatPromptValue(messages=[SystemMessage(content='あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：', additional_kwargs={}, response_metadata={}), HumanMessage(content='簡単な Hello World のコードを作成してください。', additional_kwargs={}, response_metadata={})])

ChatPromptTemplate によって ChatPromptValue のインスタンス (Message のリスト) が出力されるので、それを Chat Medel の入力として使用することができる。



In [18]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
result = model.invoke(messages)
print(result.content)

もちろんです！Pythonでの簡単な「Hello, World!」プログラムは以下のようになります。

```python
print("Hello, World!")
```

このコードを実行すると、コンソールに「Hello, World!」と表示されます。


HumanMessage だけにまとめてしままう場合は、`from_template` メソッドを使用することもできる。  
この場合は、コンテキストも HumanMessage のなかに含まれる。

In [19]:
chat_prompt = ChatPromptTemplate.from_template("""あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：

{question}""")

messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(messages)

messages=[HumanMessage(content='あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：\n\n簡単な Hello World のコードを作成してください。', additional_kwargs={}, response_metadata={})]


In [20]:
result = model.invoke(messages)
print(result.content)

もちろんです！Pythonでの簡単な「Hello, World!」プログラムは以下のようになります。

```python
print("Hello, World!")
```

このコードを実行すると、コンソールに「Hello, World!」と表示されます。Pythonの基本的な出力方法を示すシンプルな例です。


## Output parsers
https://python.langchain.com/docs/concepts/#output-parsers

### StrOutputParser
Chat model の出力をテキストに変換する (AIMessage の `content` だけを抽出する) Output parser
[langchain_core.output_parsers.string.StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

In [21]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [22]:
answer = parser.invoke(result)
print(answer)

もちろんです！Pythonでの簡単な「Hello, World!」プログラムは以下のようになります。

```python
print("Hello, World!")
```

このコードを実行すると、コンソールに「Hello, World!」と表示されます。Pythonの基本的な出力方法を示すシンプルな例です。


# Section 2: Chain と LCEL
---

ここまでのコードの書き方では、各ステップの結果物を変数に入れ次のステップに渡す必要があり、コードの量が増えてしまう。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model のインスタンスを作成
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt template のインスタンスを作成
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])
# 完全なプロンプトを作る
messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください。"})

# Output parser のインスタンスを作成
parser = StrOutputParser()

# Chat model にプロンプトを入力して結果を得る
result = model.invoke(messages)

# Chat model の出力を Output parser に入力して変換する
answer = parser.invoke(result)
print(answer)

LangChain では、その名が示す通り、Chain という形で複数のステップをつなぎ合わせて実行することができる。
Chain 同士をつなぎ合わせて Chain を作ることもできるため、単にモデルから出力を得て終わりではなく、処理を連鎖的につないで実行することができる。

## Runnables
* LangChain の主要コンポーネントは Runnable プロトコル (インターフェース) を実装しており、いくつかの共通のメソッドで操作できるようになっている。
* Runnable プロトコルを実装しているコンポーネントのオブジェクト自体を Runnable と呼ぶこともある。。



In [23]:
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])
messages = chat_prompt.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(messages)

messages=[SystemMessage(content='あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください：', additional_kwargs={}, response_metadata={}), HumanMessage(content='簡単な Hello World のコードを作成してください。', additional_kwargs={}, response_metadata={})]


In [ ]:
result = model.invoke(messages)
print(result)

In [ ]:
answer = parser.invoke(result)
print(answer)

In [ ]:
chain = chat_prompt | model | parser
answer = chain.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(answer)

## LCEL (LangChain Expression Language) による Chain の記述

*   現在の LangChain では LCEL という記法でコードを記述することが推奨されている。
*   LCEL では Runnable をパイプ (`|`) でつないで Chain を記述することができる。
*   Chain もまた Runnable であり、`invoke` メソッドで実行することができる。



先ほどのコードは LCEL を使って以下のよう書くことができる。

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model のインスタンスを作成
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt template のインスタンスを作成
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたはプログラミング言語 Python の高度なスキルをもつプロフェッショナルのエンジニアです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser のインスタンスを作成
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行する
answer = chain.invoke({"question": "簡単な Hello World のコードを作成してください。"})
print(answer)

もちろんです！Pythonでの簡単な「Hello, World!」プログラムは以下のようになります。

```python
print("Hello, World!")
```

このコードを実行すると、コンソールに「Hello, World!」と表示されます。


## Streaming による出力
*   モデルが長い文章を回答する場合、モデルが最後まで回答し終えてから出力すると、ユーザーしばらく待たされることになりユーザビリティが下がる。
*   streaming で出力することにより、モデルからの回答を順次出力していくことが可能になる。
*   Runnables は `stream` や `astream` メソッドをサポートしており、これを使用するとでモデルの回答を streaming で出力することができる。

https://python.langchain.com/docs/tutorials/chatbot/#streaming


### streamning しない場合


In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# プロンプトのテンプレートを作る
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは AI や機械学習の高度なスキルをもつプロフェッショナルのデータサイエンティストです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行する
answer = chain.invoke({"question": "AI と機械学習、ディープラーニングの関係性をわかりやすく1000文字程度で説明してください。"})
print(answer)

AI（人工知能）、機械学習（ML）、ディープラーニング（DL）は、相互に関連しながらも異なる概念です。これらの関係性を理解することは、現代の技術の進化を把握する上で非常に重要です。

まず、AI（Artificial Intelligence）とは、人間の知能を模倣することを目的とした技術の総称です。AIは、問題解決、学習、推論、計画、自然言語処理など、さまざまな知的作業を行うことができます。AIは大きく分けて、ルールベースのシステム（従来のプログラミングに基づく）と、機械学習を用いたシステムに分類されます。

次に、機械学習（Machine Learning）は、AIの一分野であり、データから学習し、経験を通じてパフォーマンスを向上させるアルゴリズムや技術を指します。機械学習は、明示的にプログラムされていないタスクを実行するために、データを使用してモデルを訓練します。機械学習には、教師あり学習、教師なし学習、強化学習などの手法があります。教師あり学習では、ラベル付きデータを用いてモデルを訓練し、教師なし学習では、ラベルなしデータからパターンを見つけ出します。

さらに、ディープラーニング（Deep Learning）は、機械学習の一種であり、特に神経ネットワークを用いた手法です。ディープラーニングは、多層のニューラルネットワークを使用して、データの特徴を自動的に抽出し、複雑なパターンを学習します。これにより、画像認識、音声認識、自然言語処理などの分野で非常に高い精度を達成しています。ディープラーニングは、大量のデータと計算リソースを必要としますが、その結果として得られる性能は非常に優れています。

要約すると、AIは知能を模倣する広範な分野であり、その中に機械学習が位置し、さらにその中にディープラーニングが存在します。AIは人間の知能を再現するための全体的な枠組みであり、機械学習はその実現手段の一つ、ディープラーニングはその中でも特に強力な手法です。このように、AI、機械学習、ディープラーニングは階層的な関係にあり、それぞれが異なる役割を果たしています。これらの技術は、今後も進化し続け、さまざまな分野での応用が期待されています。


### streaming で出力する場合
[How to stream runnables](https://python.langchain.com/docs/how_to/streaming/)


In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# プロンプトのテンプレートを作る
chat_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "あなたは AI や機械学習の高度なスキルをもつプロフェッショナルのデータサイエンティストです。次の質問に答えてください："
    ),
    (
        "human",
        "{question}"
    )
])

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# Chain を実行して streaming で出力する
for chunk in chain.stream({"question": "AI と機械学習、ディープラーニングの関係性をわかりやすく1000文字程度で説明してください。"}):
  print(chunk, end='', flush=True)


AI（人工知能）、機械学習（ML）、ディープラーニング（DL）は、相互に関連しながらも異なる概念です。これらの関係性を理解することは、現代の技術の進化を把握する上で非常に重要です。

まず、AI（Artificial Intelligence）とは、人間の知能を模倣することを目的とした技術やシステムの総称です。AIは、問題解決、学習、推論、計画、自然言語処理など、さまざまな知的作業を行うことができます。AIは大きく分けて、ルールベースのシステム（従来のプログラミングに基づく）と、機械学習を用いたシステムに分類されます。

次に、機械学習（Machine Learning）は、AIの一分野であり、データから学習し、経験を通じてパフォーマンスを向上させるアルゴリズムや手法を指します。機械学習は、明示的にプログラムされるのではなく、データを用いてモデルを訓練し、予測や分類を行います。機械学習には、教師あり学習、教師なし学習、強化学習などの手法があります。教師あり学習では、ラベル付きデータを用いてモデルを訓練し、教師なし学習では、ラベルのないデータからパターンを見つけ出します。

さらに、ディープラーニング（Deep Learning）は、機械学習の一種であり、特に神経ネットワークを用いた手法です。ディープラーニングは、多層のニューラルネットワークを使用して、データの特徴を自動的に抽出し、複雑なパターンを学習します。これにより、画像認識、音声認識、自然言語処理などのタスクで非常に高い精度を達成することが可能です。ディープラーニングは、大量のデータと計算リソースを必要としますが、その結果として得られる性能は非常に優れています。

要約すると、AIは知能を模倣する広範な分野であり、その中に機械学習が位置し、さらにその中にディープラーニングが存在します。AIは人間の知能を再現するための全体的な枠組みであり、機械学習はその実現手段の一つ、ディープラーニングはその中でも特に強力な手法です。これらの技術は、今後の技術革新や社会の変革に大きな影響を与えると期待されています。

## LCEL を理解するための How-to ガイド
https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel

# Secton 3: Chat history
---
https://python.langchain.com/docs/concepts/#chat-history



*   ここまではモデルとの一問一答での対話だったが、チャットでは会話の継続性が必要になる。
*   ただ、生成 AI モデルの API はステートレスであり、モデル側に会話の履歴を記憶させることはできない。
*   しかし、Chat history を使用して会話の履歴をプロンプトに追加していくことで、会話に継続性をもたせることができるようになる。



### 会話履歴がない場合

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chat template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

以下のセルを実行して、以前の入力や回答に言及する継続的な会話をしてみましょう。

In [28]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = chain.invoke({"input": q})
    print(f'\nAnswer: {answer}')

Write Quit, Exit or Bye to quit.
Your prompt: 僕は

Answer: こんにちは！何かお話ししたいことがありますか？
Your prompt: ケイです

Answer: こんにちは、ケイさん！今日はどんなことをお話ししましょうか？
Your prompt: 嬉しです

Answer: それは良かったですね！何が嬉しいのですか？お話を聞かせてください。
Your prompt: 名前を覚えてもらったこと

Answer: もちろんです！あなたの名前を教えていただければ、会話の中で使うことができますよ。どうぞお知らせください。
Your prompt: 僕は何を嬉しがった？

Answer: あなたが嬉しがったことについて具体的な情報はわかりませんが、何か特別な出来事や成功、楽しい経験があったのかもしれませんね。最近の出来事や感じたことを教えてもらえれば、もっとお話しできるかもしれません！


KeyboardInterrupt: Interrupted by user

## LangChain の記憶 (会話履歴) に関する機能

### Chat History - BaseChatMessageHistory を継承したクラスたち
[langchain_community.chat_message_histories](https://python.langchain.com/api_reference/community/chat_message_histories.html)  

#### 例：
`ChatMessageHistory`, `SQLChatMessegeHistory`, `RedisChatMessageHistory`, `DynamoDBChatMessageHistory`, etc.  


#### 概要：
*  決まった形式で会話履歴をデータベース (やメモリ) に保存する機能を提供する。
*  裏で会話履歴の Message を保存するリストを持っていて、そのリストに対する Message の追加や読み出しができる。
*  データベースに読み書きする機能がクラスに実装されているため、その部分のコードを書く必要がなく、バックエンドのデータベースが異なっていてもコードとしてはほぼ同じインターフェイスで操作できる。
*  RDB で自前でテーブル設計して会話履歴を保存したい場合は、これらのクラスが実行している処理を自前で実装すればよい。


#### 使い方：
*  `add_user_message`/`add_ai_message` メソッドを使って、自前で履歴を管理する
*  `RunnableWithMessageHistory` と合わせて使用する

---
### Memory - BaseMemory を継承したクラスたち
[langchain.memory](https://python.langchain.com/api_reference/langchain/memory.html)

#### 例：
`ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationSummaryMemory`, etc.

#### 概要：
*  「BaseChatMessageHistory を継承したクラスたち」に対して、会話履歴の一部だけを使用したり、履歴を要約してプロンプトに使用する等の機能を提供する。
*  v0.2 のドキュメントには記載がない (v0.1 のドキュメントには [Beta] と記載されている)


## 会話履歴による会話の継続性の基本的な考え方
ユーザーによる最新の入力だけでなく、過去の会話履歴の Messages をプロンプトに入れてモデルに渡す。  
参考：[How to add memory to chatbots](https://python.langchain.com/docs/how_to/chatbots_memory/)

In [29]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chat template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは人間と会話するチャットボットです。",
        ),
        MessagesPlaceholder(variable_name="messages"), # 複数の Message をプロンプトに挿入するためのプレースホルダ
    ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義
chain = chat_prompt | model | parser

# Chain を実行
answer = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="私の名前はジョンといいます。"
            ),
            AIMessage(content="ジョンさんですね。よろしくお願いします。"),
            HumanMessage(content="私の名前がわかりますか。"),
        ],
    }
)
print(answer)

はい、ジョンさんとおっしゃっていましたね。何かお話ししたいことがありますか？


## Chat history を使う
まず、`ChatMessageHistory` がどのように会話履歴を保持するか確認する。

In [30]:
from langchain_community.chat_message_histories import ChatMessageHistory

# ChatMessageHistory のインスタンスを作成
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message(
    "私の名前はジョンといいます。"
)

demo_ephemeral_chat_history.add_ai_message("ジョンさんですね。よろしくお願いします。")

demo_ephemeral_chat_history.messages

[HumanMessage(content='私の名前はジョンといいます。', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ジョンさんですね。よろしくお願いします。', additional_kwargs={}, response_metadata={})]

*  `ChatMessageHistory` は `add_user_message` や `add_ai_message` というメソッドを備えており、これを使って会話履歴の Message を追加保存していくことができる。
*  追加した Message は、Message のリストとして保持される。

ユーザーの入力と、それに対するモデルからの回答を順次追加していき、それをプロンプトに含めることによって、会話履歴をもった継続性のある会話が可能となる。

In [31]:
demo_ephemeral_chat_history = ChatMessageHistory()

# 1 番目の入力
input1 = "私の名前はジョンといいます。"
print(f"INPUT1: {input1}")

demo_ephemeral_chat_history.add_user_message(input1)

# 1 番目の入力に対して回答を出力
response = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)
print(f"ANSWER1: {response}")

# 1 番目の回答を Chat history に追加
demo_ephemeral_chat_history.add_ai_message(response)

# 2 番目の入力
input2 = "私の名前がわかりますか。"
print(f"INPUT2: {input2}")

demo_ephemeral_chat_history.add_user_message(input2)

# 2 番目の入力に対して回答を出力
answer = chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
    }
)
print(f"ANSWER2: {answer}")

INPUT1: 私の名前はジョンといいます。
ANSWER1: ジョンさん、こんにちは！お話しできてうれしいです。今日はどんなことをお話ししましょうか？
INPUT2: 私の名前がわかりますか。
ANSWER2: はい、ジョンさんという名前だとおっしゃっていましたね。何か特別なことについてお話ししたいことがありますか？


## Chat history を RunnableWithMessageHistory と組み合わせて使用する
*  `RunnableWithMessageHistory` は別の Runnable のために Chat history を管理してくれる Runnable 。
*  別の Runnable (≒ Chain) をラップして、その Chat history の読み出しや更新を行う。
*  `add_user_message` や `add_ai_message` を使ってコードを書かずに済むが、抽象化されてしまっているためややわかりにくく、コードの書き方もクラスの制約を受ける。

[langchain_core.runnables.history.RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)

In [32]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 特定のセッション ID に対する ChatMessageHistory のインスタンスを作成
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# Prompt template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      MessagesPlaceholder(variable_name="chat_history"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# RunnableWithMessageHistory のインスタンスを作成
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [33]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = runnable_with_history.invoke({"input": q}, config={"configurable": {"session_id": "a123"}})
    print(f'\nAnswer: {answer}')

Write Quit, Exit or Bye to quit.
Your prompt: 僕の名前は

Answer: あなたの名前は何ですか？教えていただければ嬉しいです！
Your prompt: としきです

Answer: としきさん、こんにちは！お話しできて嬉しいです。今日はどんなことを話しましょうか？
Your prompt: 私の名前わかりますか

Answer: はい、としきさんですね！何か特別なことについてお話ししたいことがありますか？
Your prompt: quit
Quitting ... bye bye!


## SQLChatMessageHistory を使用する
[langchain_community.chat_message_histories.sql.SQLChatMessageHistory](https://python.langchain.com/api_reference/community/chat_message_histories/langchain_community.chat_message_histories.sql.SQLChatMessageHistory.html)  
参考：[How to add message history](https://python.langchain.com/docs/how_to/message_history/)

In [34]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_message_histories.sql import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Chat model を用意する
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 特定のセッション ID に対する SQLChatMessageHistory のインスタンスを作成
def get_session_history_sql(session_id):
    return SQLChatMessageHistory(session_id, connection_string="sqlite:///memory.db")

# Prompt template を用意する
chat_prompt = ChatPromptTemplate.from_messages(
  [
      SystemMessagePromptTemplate.from_template("あなたは人間と会話するチャットボットです"),
      MessagesPlaceholder(variable_name="chat_history"),
      HumanMessagePromptTemplate.from_template("{input}")
  ]
)

# Output parser を用意する
parser = StrOutputParser()

# Chain を定義する
chain = chat_prompt | model | parser

# RunnableWithMessageHistory のインスタンスを作成
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history_sql,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [35]:
print('Write Quit, Exit or Bye to quit.')
while True:
    q = input('Your prompt: ')
    if q.lower() in ['quit', 'exit', 'bye']:
        print('Quitting ... bye bye!')
        break

    answer = runnable_with_history.invoke({"input": q}, config={"configurable": {"session_id": "a456"}})
    print(f'\nAnswer: {answer}')

Write Quit, Exit or Bye to quit.
Your prompt: こんにちは


/usr/local/lib/python3.11/dist-packages/langchain_core/runnables/history.py:600: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(



Answer: こんにちは！今日はどんなことをお話ししましょうか？
Your prompt: さっき僕なんて言った？

Answer: 「こんにちは」と言いましたね。何か特別なことをお話ししたいですか？


KeyboardInterrupt: Interrupted by user

In [ ]:
! rm memory.db

Chat history の各クラスを利用して、様々なデータベースに会話履歴を保存することができます。
https://python.langchain.com/docs/integrations/memory/  
(ドキュメントのタイトルは Memory ですが、記載されている内容は Chat history です)